# Week 12-13 Data Warehouse & ETL

This week and next we'll build up a mini warehouse using Bikeshare trip data and weather data.

We are going to use [PostgreSQL](https://www.postgresql.org) 12 or later version. PostgreSQL is already installed on your AWS EC2 instances based on our AMI.

## Setup - bikeshare data, again

We'll download the same Bikeshare data you've worked with before, and we'll create some database tables using PostgreSQL.

In [ ]:
%load_ext sql

In [ ]:
!dropdb -U student week13

In [ ]:
!createdb -U student week13

In [ ]:
%sql postgresql://student@/week13

Loading capital bikeshare trip data for 2017

In [ ]:
!wget -O 2017-Q1-trips.zip https://s3.amazonaws.com/capitalbikeshare-data/2017-capitalbikeshare-tripdata.zip

In [ ]:
!unzip -o 2017-Q1-trips.zip

In [ ]:
!csvstack 2017Q*-capitalbikeshare-tripdata.csv >  2017q1-4.csv

In [ ]:
!wc -l 2017q1-4.csv

In [ ]:
!csvcut -n 2017q1-4.csv

In [ ]:
!rm -f 2017Q*.csv

In [ ]:
!rm -f 2017-Q1-trips.zip

### Create table and import

Given the volume of data here, let's go straight to pgsql to load the data.

In [ ]:
!head -n 10000 2017q1-4.csv | csvstat

Based on these values, I expect we can work with the following:

In [ ]:
%%sql
DROP TABLE IF EXISTS rides;

CREATE TABLE rides (
    duration_ms INTEGER NOT NULL,
    start_date TIMESTAMP NOT NULL,
    end_date TIMESTAMP NOT NULL,
    start_station_id INTEGER NOT NULL,
    start_station VARCHAR(64) NOT NULL,
    end_station_id INTEGER NOT NULL,
    end_station VARCHAR(64) NOT NULL,
    bike_number VARCHAR(6) NOT NULL,
    member_type VARCHAR(6) NOT NULL
)

In [ ]:
%%sql
SELECT * from rides limit 5

Now we'll load the data directly using `COPY` command.  Note that this **requires** the use of an absolute path, so adjust it to your location:

In [ ]:
!pwd

In [ ]:
%%sql
COPY rides FROM '/home/ubuntu/Dimensional-modeling/2017q1-4.csv'
CSV
HEADER;

In [ ]:
%%sql
SELECT COUNT(*) FROM rides;

In [ ]:
!wc -l 2017q1-4.csv

In [ ]:
%%sql
SELECT * FROM rides
LIMIT 5

In [ ]:
%%sql
SELECT length(bike_number) l_bike, count(1)
FROM rides
where length(bike_number) > 6
GROUP BY length(bike_number)

In [ ]:
%%sql
SELECT bike_number
FROM rides
where length(bike_number) > 6

In [ ]:
%%sql
DELETE from rides where length(bike_number)>6

## More ETL with SQL

Today we will start with this schema: 

In [ ]:
from IPython.display import Image
Image(url="https://github.com/aliobaidi/datamanagement/blob/baa3a3a3c1b23ce104324e284ddf46b903bce6e7/rides.png?raw=true")  

We will eventually build a star schema like this:

In [ ]:
Image(url="https://github.com/aliobaidi/datamanagement/blob/master/rides_star.png?raw=true")

# Work on Station dimension, modify fact and link them together
First let's extract simple details like station ids and names.

In [ ]:
%%sql
SELECT DISTINCT start_station_id, start_station
FROM rides
ORDER BY start_station
LIMIT 10;

In [ ]:
%%sql
SELECT DISTINCT end_station_id, end_station
FROM rides
ORDER BY end_station
LIMIT 10;

To be sure we get them all, we need to combine them into a union set.

In [ ]:
%%sql
SELECT DISTINCT start_station_id AS station_id, 
       start_station AS station 
FROM rides
UNION
SELECT DISTINCT end_station_id AS station_id, 
       end_station AS station 
FROM rides
Limit 10

Now we can create a new dimension table to house the unique station ids and names.

In [ ]:
%%sql
DROP TABLE IF EXISTS station;

CREATE TABLE station (
    key SERIAL PRIMARY KEY,
    station_id INTEGER,
    name VARCHAR(64)
);

In [ ]:
%%sql
INSERT INTO station (station_id, name)
SELECT DISTINCT start_station_id AS station_id, 
       start_station AS station 
FROM rides
UNION
SELECT DISTINCT end_station_id AS station_id, 
       end_station AS station 
FROM rides;

In [ ]:
%%sql
SELECT * FROM station LIMIT 10;

We add these new identifiers (surrogate key) back to the fact table.

In [ ]:
%%sql
ALTER TABLE rides
ADD COLUMN start_station_key INTEGER,
ADD CONSTRAINT fk_start_station
    FOREIGN KEY (start_station_key)
    REFERENCES station (key);

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN end_station_key INTEGER,
ADD CONSTRAINT fk_end_station
    FOREIGN KEY (end_station_key)
    REFERENCES station (key);

In [ ]:
%%sql 
SELECT * FROM rides LIMIT 10;

In [ ]:
%%sql
UPDATE rides
SET start_station_key = station.key
FROM station
WHERE rides.start_station = station.name;

In [ ]:
%%sql 
SELECT * FROM rides LIMIT 10;

Now we update the `start_station_key` in the fact table with the values from station dimension table based on the station ids (natural key).

In [ ]:
%%sql
UPDATE rides
SET end_station_key = station.key
FROM station
WHERE rides.end_station_id = station.station_id;

Now we can drop the attributes `start_station_id`, `start_station`, `end_station_id` and `end_station` from the rides relation.

In [ ]:
%%sql 
SELECT * FROM rides LIMIT 10;

#### adding more context to the station dimension table (Latitude and Longitude)

In [ ]:
%%sql
ALTER TABLE rides 
DROP COLUMN start_station_id,
DROP COLUMN start_station,
DROP COLUMN end_station_id,
DROP COLUMN end_station;

In [ ]:
%%sql
SELECT * 
FROM rides
LIMIT 10;

### Simple address geocoding

It feels like we should do a little more with the stations, doesn't it?  Let's see if we can geocode them using the [geocoder library](https://geocoder.readthedocs.io/). To install the `geocoder` package, use : 

Code to install Geocoder is !pip install --user geocoder

In [ ]:
%%sql
ALTER TABLE station
ADD COLUMN lat NUMERIC,
ADD COLUMN lng NUMERIC;

In [ ]:
%%sql
SELECT *
FROM station
Limit 10;

#### Connecting to the db from python

Here we'll use a little python code to get the geographic coordinates based on station's names. Since it may take a while to geocode all 452 stations, we will only geocode 10 stations in the class.

In [ ]:
import psycopg2
import geocoder

conn = psycopg2.connect("dbname='week13' user='student'")
c = conn.cursor()
c.execute("SELECT key, name FROM station ORDER BY key ASC")
rows = c.fetchall()
for r in rows:
    station_key, station_name = r
    print('%s: %s' % (station_key, station_name))
    g = geocoder.arcgis('%s, Washington DC' % station_name)
    if g:
        c.execute("UPDATE station SET lat = (%s), lng = (%s) WHERE key = (%s)", 
                  (g.lat, g.lng, station_key))
    else:
        print("no geocode")
conn.commit()

In [ ]:
%%sql
SELECT * FROM station ORDER BY key ASC LIMIT 10;

In [ ]:
%%sql
SELECT * 
FROM station
Where lat between 38 and 39 and
      lng between -77 and -75 

In [ ]:
%%sql
--Just a Reminder of what we have done so far
SELECT * from rides
Limit 10;

We want to build the dimension table time. We assume the aggregate level is per hour. 
We want to also add whether a day falls on a weekend or not and for which week and 
quarter of the year

## Add more derived facts and dimensions

Another useful step might be recording the minutes as a new column so we don't have to calculate from milliseconds every time.

Let's experiment with the start date

In [ ]:
%%sql
ALTER TABLE rides
ADD COLUMN duration_min NUMERIC;

In [ ]:
%%sql
UPDATE rides
SET duration_min = ROUND(CAST(duration_ms AS NUMERIC) / (60), 2);

In [ ]:
%%sql
SELECT * FROM rides
order by duration_min desc
LIMIT 5;

In data warehouse models and in statistical model feature engineering, it can be particularly useful to extract all kinds of parts of dates out into their own attributes.  You never know where you'll find significance.

This kind of extraction is quite common.

In [ ]:
%%sql
Alter table rides drop column duration_ms

In [ ]:
%%sql
SELECT DISTINCT start_date, 
    TO_CHAR(start_date, 'YYYY-MM-DD HH24:00:00') AS hour,
    TO_CHAR(start_date, 'YYYY-MM-DD') AS day, 
    TO_CHAR(start_date, 'YYYY') AS year,
    TO_CHAR(start_date, 'Month') AS month_of_year_str,
    TO_CHAR(start_date, 'MM') AS month_of_year,
    TO_CHAR(start_date, 'DD') AS day_of_month,
    TO_CHAR(start_date, 'Day') AS day_of_week_str,
    TO_CHAR(start_date, 'D') AS day_of_week,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) in (7,1)
        THEN 'true' 
        ELSE 'false'
    END AS is_weekend,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) Not in (7,1) 
        THEN 'true' 
        ELSE 'false'
    END AS is_weekday,
    TO_CHAR(start_date, 'HH24') AS hour_of_day,
    TO_CHAR(start_date, 'Q') AS quarter_of_year
FROM rides
LIMIT 10;

#### Create the Time Dimension Table

In [ ]:
%%sql
DROP TABLE IF EXISTS hour;

CREATE TABLE hour (
    key SERIAL PRIMARY KEY,
    hour CHAR(19),
    day CHAR(10),
    year INTEGER,
    month_of_year_str VARCHAR(12),
    month_of_year INTEGER,
    day_of_month INTEGER,
    day_of_week_str CHAR(9),
    day_of_week INTEGER,
    is_weekend BOOLEAN,
    is_weekday BOOLEAN,
    hour_of_day INTEGER,
    quarter_of_year INTEGER
);

In [ ]:
%%sql
INSERT INTO hour (hour, day, year, month_of_year_str, month_of_year, day_of_month, 
                  day_of_week_str, day_of_week, is_weekend, is_weekday, 
                  hour_of_day, quarter_of_year)
SELECT DISTINCT TO_CHAR(start_date, 'YYYY-MM-DD HH24:00:00') AS hour,
    TO_CHAR(start_date, 'YYYY-MM-DD') AS day, 
    CAST(TO_CHAR(start_date, 'YYYY') AS INTEGER) AS year,
    TO_CHAR(start_date, 'Month') AS month_of_year_str,
    CAST(TO_CHAR(start_date, 'MM') AS INTEGER) AS month_of_year,
    CAST(TO_CHAR(start_date, 'DD') AS INTEGER) AS day_of_month,
    TO_CHAR(start_date, 'Day') AS day_of_week_str,
    CAST(TO_CHAR(start_date, 'D') AS INTEGER) AS day_of_week,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekend,
    CASE WHEN CAST(TO_CHAR(start_date, 'D') AS INTEGER) NOT IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekday,
    CAST(TO_CHAR(start_date, 'HH24') AS INTEGER) AS hour_of_day,
    CAST(TO_CHAR(start_date, 'Q') AS INTEGER) AS quarter_of_year
FROM rides
UNION
SELECT DISTINCT TO_CHAR(end_date, 'YYYY-MM-DD HH24:00:00') AS hour,
    TO_CHAR(end_date, 'YYYY-MM-DD') AS day, 
    CAST(TO_CHAR(end_date, 'YYYY') AS INTEGER) AS year,
    TO_CHAR(end_date, 'Month') AS month_of_year_str,
    CAST(TO_CHAR(end_date, 'MM') AS INTEGER) AS month_of_year,
    CAST(TO_CHAR(end_date, 'DD') AS INTEGER) AS day_of_month,
    TO_CHAR(end_date, 'Day') AS day_of_week_str,
    CAST(TO_CHAR(end_date, 'D') AS INTEGER) AS day_of_week,
    CASE WHEN CAST(TO_CHAR(end_date, 'D') AS INTEGER) IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekend,
    CASE WHEN CAST(TO_CHAR(end_date, 'D') AS INTEGER) NOT IN (1, 7) 
        THEN TRUE
        ELSE FALSE
    END AS is_weekday,
    CAST(TO_CHAR(end_date, 'HH24') AS INTEGER) AS hour_of_day,
    CAST(TO_CHAR(end_date, 'Q') AS INTEGER) AS quarter_of_year
FROM rides;

In [ ]:
%%sql
SELECT * FROM hour
LIMIT 10;

And let's make sure we got that weekend bit right:

In [ ]:
%%sql
SELECT DISTINCT day_of_week_str, day_of_week, is_weekend, is_weekday 
FROM hour
ORDER BY day_of_week;

We now add FK `start_hour_key` to the fact table.

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN start_hour_key INTEGER,
ADD CONSTRAINT fk_start_hour
    FOREIGN KEY (start_hour_key)
    REFERENCES hour (key);

Now we update the `start_hour_key` in the fact table with the values from hour dimension table based on start time of the trips.

In [ ]:
%%sql
UPDATE rides
SET start_hour_key = hour.key
FROM hour
WHERE TO_CHAR(rides.start_date, 'YYYY-MM-DD HH24:00:00') = hour.hour;

Do the same for the `end_hour_key`:

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN end_hour_key INTEGER,
ADD CONSTRAINT fk_end_hour
    FOREIGN KEY (end_hour_key)
    REFERENCES hour (key);

In [ ]:
%%sql
UPDATE rides
SET end_hour_key = hour.key
FROM hour
WHERE TO_CHAR(rides.end_date, 'YYYY-MM-DD HH24:00:00') = hour.hour;

In [ ]:
%%sql
SELECT * FROM rides
LIMIT 5;

Let's verify that those key values are correct. Here is the query that returns the first 10 trips of the bike W20893, with start/end dates as well as the start/end hours. It joins the rides fact table with hour dimension table:

In [ ]:
%%sql
SELECT rides.start_date, rides.end_date, s_hour.hour AS start_hour, e_hour.hour AS end_hour
FROM rides
JOIN hour AS s_hour
  ON s_hour.key = rides.start_hour_key
JOIN hour AS e_hour
  ON e_hour.key = rides.end_hour_key
WHERE bike_number = 'W20893'
ORDER BY rides.start_date
LIMIT 10;

In [ ]:
%%sql
SELECT * from rides limit 5

We can drop the `start_date` and `end_date` if we are not interested at the minute level.

In [ ]:
%%sql
alter table rides drop column start_date, drop column end_date

#### Let's create member_type and bike dimensions:

In [ ]:
%%sql
DROP TABLE IF EXISTS member_type;

CREATE TABLE member_type (
    key SERIAL PRIMARY KEY,
    member_type CHAR(6)
);

DROP TABLE IF EXISTS bike;

CREATE TABLE bike (
    key SERIAL PRIMARY KEY,
    bike_number CHAR(6)
);

We populate the member_type dimension table. As you can see there is only two rows in this dimension table:

In [ ]:
%%sql
INSERT INTO member_type (member_type)
SELECT DISTINCT member_type 
FROM rides;

Next, we add FK `member_type_key` to the rides table:

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN member_type_key INTEGER,
ADD CONSTRAINT fk_member_type
    FOREIGN KEY (member_type_key)
    REFERENCES member_type (key);

...and populate the member type FK:

In [ ]:
%%sql
UPDATE rides AS r
SET member_type_key = m.key
FROM member_type AS m
WHERE r.member_type = m.member_type;

Same steps for the bike:

In [ ]:
%%sql
INSERT INTO bike (bike_number)
SELECT DISTINCT bike_number 
FROM rides;

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN bike_key INTEGER,
ADD CONSTRAINT fk_bike
    FOREIGN KEY (bike_key)
    REFERENCES bike (key);

In [ ]:
%%sql
UPDATE rides
SET bike_key = bike.key
FROM bike
WHERE rides.bike_number = bike.bike_number;

In [ ]:
%%sql
SELECT * FROM rides
LIMIT 5;

We can drop the `bike_number` and `member_type` from the rides table.

In [ ]:
%%sql
ALTER TABLE rides 
DROP COLUMN bike_number,
DROP COLUMN member_type;

Now look at the final fact table:

In [ ]:
%%sql
SELECT start_station_key, end_station_key, start_hour_key, end_hour_key,
member_type_key, bike_key, duration_min 
FROM rides
LIMIT 5;

# Data Exploration 
Now we are ready to explore the data. 
Let's have an example of bike that made more than 40 trips. Indicate the start station name 

In [ ]:
%%sql
SELECT 
 s.name, b.bike_number,
 count(1) as "number of trips",
 sum(duration_min) sum_duration_min,
 avg(duration_min) avg_duration_min
FROM
 Station S, Bike B, Rides R
Where
 s.key=r.start_station_key and 
 b.key=r.bike_key 
Group by 
 s.name, b.bike_number
Having 
 Count(1)>40
Order by 
 count(1) desc


Lets find the number of trips per day

In [ ]:
%%sql
SELECT day_of_week_str, COUNT(*) count
FROM rides
JOIN hour
  ON rides.start_hour_key = hour.key
GROUP BY day_of_week_str, day_of_week
ORDER BY day_of_week;

Let's make use of the analytic (window) functions
function_name(*) Over (partition by expression order by expression ASC/Desc)

count(*) over (partition by day_of_week_str order by day_of_week)

In [ ]:
%%sql
select distinct day_of_week, day_of_week_str, count(*) over (partition by day_of_week_str order by day_of_week)
from rides join hour on rides.start_hour_key = hour.key
order by day_of_week

In [ ]:
%matplotlib inline

In [ ]:
_.bar()

Let's look at the trips separately from casual riders and registered (members) riders:

In [ ]:
%%sql
SELECT day_of_week_str, COUNT(*) count
FROM rides
JOIN hour
  ON rides.start_hour_key = hour.key
JOIN member_type
    ON rides.member_type_key = member_type.key
WHERE member_type.member_type = 'Casual'
GROUP BY day_of_week_str, day_of_week
ORDER BY day_of_week;

In [ ]:
_.bar()

In [ ]:
%%sql
SELECT day_of_week_str, COUNT(*) count
FROM rides
JOIN hour
  ON rides.start_hour_key = hour.key
JOIN member_type
    ON rides.member_type_key = member_type.key
WHERE member_type.member_type = 'Member'
GROUP BY day_of_week_str, day_of_week
ORDER BY day_of_week;

In [ ]:
_.bar()

Let's check the count for each hour of the day

In [ ]:
%%sql
SELECT hour.hour_of_day, COUNT(*) count
FROM rides
JOIN hour
    ON rides.start_hour_key = hour.key
JOIN member_type
    ON rides.member_type_key = member_type.key
WHERE member_type.member_type = 'Casual'
GROUP BY hour.hour_of_day 
ORDER BY hour.hour_of_day;

In [ ]:
_.bar()

In [ ]:
%%sql
SELECT hour.hour_of_day, COUNT(*) count
FROM rides
JOIN hour
    ON rides.start_hour_key = hour.key
JOIN member_type
    ON rides.member_type_key = member_type.key
WHERE member_type.member_type = 'Member'
GROUP BY hour.hour_of_day 
ORDER BY hour.hour_of_day;

In [ ]:
_.bar()

In [ ]:
%%sql
select * from member_type

## Adding weather data

An interesting dimension to the bikeshare history is weather - I know I don't mind riding in the rain, but I'm probably in the minority.

Weather Underground offers access to weather history data at links like https://www.wunderground.com/history/airport/KDCA/2017/1/18/DailyHistory.html. Until recently we were able to download the weather history data in CSV format for free. Now Weather Underground offers API access that supports JSON and XML formats only and starts to charge for it based on usage. 

Fortunately we have downloaded the data for 2017 Q1 before they disabled the CSV suppport. There is one CSV file for each day. You can download them all as a zip file.

In [ ]:
%%sql
SELECT * from pg_tables where upper(tableowner)='STUDENT';

In [ ]:
!wget -O weather2017q1.csv.zip https://s3.amazonaws.com/dmfa-2017/weather2017q1.csv.zip

In [ ]:
!unzip weather2017q1.csv.zip

In [ ]:
!head weather-20170125.csv | csvlook

Something is not right! The header is missing and there is `<br />` at the end of each line. Let's look at the raw content of the CSV file.

In [ ]:
!head weather-20170125.csv

There are two issues:
1. The first line is blank
2. There are extra characters at the end of each line.

Let's clean the data using sed (stream editor) command. sed command "`/^$/d`" removes blank lines. "`s/<br \/>//g`" finds and replace `<br \/>` with empty string `''`.

In [ ]:
!sed '/^$/d;s/<br \/>//g' weather-20170125.csv | head | csvlook

Now it looks much better! Apply the fix to all weather CSV files.

In [ ]:
!for f in weather-2017*.csv; do sed -i '/^$/d;s/<br \/>//g' ${f}; done

If you don't know what the -i option do, you can find it out in the help:

In [ ]:
!sed --help

And combine date based csv files into a single file:

In [ ]:
!csvstack weather-201701*.csv weather-201702*.csv weather-201703*.csv > weather-2017q1.csv

In [ ]:
!csvstat weather-2017q1.csv

We've noticed special values such as `-`, `N/A` and `None`. We need to remove them so that they will be treated as NULL by the database.

In [ ]:
!sed -i 's/,N\/A,/,,/g;s/,-,/,,/g;;s/,None,/,,/g' weather-2017q1.csv

Based on these values, I expect we can work with the following schema for weather. Note that the type for `time_utc` is `TIMESTAMPTZ`, which is the abbreviation for `timestamp with time zone`, a PostgreSQL specific type. We also add an attribute named `time`. We will use it to store local eastern time.

In [ ]:
!rm -f weather-20170*.csv

In [ ]:
%%sql
DROP TABLE IF EXISTS weather;

CREATE TABLE weather (
    key SERIAL PRIMARY KEY,
    time_str VARCHAR(8),
    temp NUMERIC,
    dew_point NUMERIC,
    humidity NUMERIC,
    pressure NUMERIC,
    visibility NUMERIC,
    wind_dir VARCHAR(8),
    wind_speed VARCHAR(10),
    gust_speed NUMERIC,
    precipitation NUMERIC,
    events VARCHAR(50),
    conditions VARCHAR(50),
    wind_dir_degrees NUMERIC,
    time_utc TIMESTAMPTZ,
    time TIMESTAMP
)

Now we'll load the data into PostgreSQL. Note that this requires the use of an absolute path, so adjust it to your location:

In [ ]:
!pwd

In [ ]:
%%sql
COPY weather 
(time_str, temp, dew_point, humidity, pressure, visibility, wind_dir, wind_speed, gust_speed, 
 precipitation, events, conditions, wind_dir_degrees, time_utc)
FROM '/home/ubuntu/Dimensional-modeling/weather-2017q1.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

In [ ]:
%%sql
SELECT * from weather LIMIT 10;

Next, we need to convert UTC time to local time EST or EDT. We know Daylight Saving Time started on Sunday, March 12, 2017, 2:00:00 am. The conversion takes two steps:

First we convert UTC times to EST times and populate `time` attribute for all `time_utc` values before `2017-03-12 07:00:00+00:00`, which is Sunday, March 12, 2017, 2:00:00 am EST. 

In [ ]:
%%sql
UPDATE weather SET time = time_utc AT TIME ZONE 'EST'
WHERE time_utc <= '2017-03-12 07:00:00+00:00';

Next we convert UTC times to EDT times and populate `time` attribute for all `time_utc` values after `2017-03-12 07:00:00+00:00`. 

In [ ]:
%%sql
UPDATE weather SET time = time_utc AT TIME ZONE 'EDT'
WHERE time_utc > '2017-03-12 07:00:00+00:00';

Verify that time attributes look okay on March 12:

In [ ]:
%%sql
SELECT time_str, time from weather 
WHERE TO_CHAR(time, 'YYYY-MM-DD') = '2017-03-12'
ORDER BY time;

Now we add two foreign key columns (`start_weather_key` and `end_weather_key`) to the `rides` table that reference `weather` dimension table.

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN start_weather_key INTEGER,
ADD CONSTRAINT fk_start_weather
    FOREIGN KEY (start_weather_key)
    REFERENCES weather (key);

In [ ]:
%%sql
ALTER TABLE rides 
ADD COLUMN end_weather_key INTEGER,
ADD CONSTRAINT fk_end_weather
    FOREIGN KEY (end_weather_key)
    REFERENCES weather (key);

In [ ]:
%%sql
UPDATE rides
SET start_weather_key = weather.key
FROM weather, hour 
WHERE rides.start_hour_key = hour.key AND hour.hour = TO_CHAR(weather.time, 'YYYY-MM-DD HH24:00:00');

In [ ]:
%%sql
UPDATE rides
SET end_weather_key = weather.key
FROM weather, hour 
WHERE rides.end_hour_key = hour.key AND hour.hour = TO_CHAR(weather.time, 'YYYY-MM-DD HH24:00:00');

Some rides do not have weather captured because there are some missing hours in the weather data:

In [ ]:
%%sql
SELECT COUNT(*) FROM rides
WHERE start_weather_key IS NULL;

In [ ]:
%%sql
SELECT COUNT(*) FROM rides
WHERE end_weather_key IS NULL;

Let's find out the top 5 weather conditions that people ride bikeshare.

In [ ]:
%%sql
SELECT w.conditions, COUNT(*) count
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
GROUP BY w.conditions
ORDER BY count DESC
LIMIT 5;

In [ ]:
%%sql
SELECT w.conditions, count(*) cnt, RANK () OVER (order by count(*) desc) rnk
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
group by w.conditions
order by rnk
limit 5;


In [ ]:
_.bar()

And the top 10 weather conditions that people ride bikeshare the least often:

In [ ]:
%%sql
SELECT w.conditions, COUNT(*) count
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
GROUP BY w.conditions
ORDER BY count ASC
LIMIT 10;

In [ ]:
%%sql
SELECT w.conditions, count(*) cnt, RANK () OVER (order by count(*) nulls first) rnk
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
group by w.conditions
order by rnk
limit 10;


In [ ]:
_.bar()

In [ ]:
%%sql
SELECT w.conditions, month_of_year, count(*) cnt, RANK () OVER (order by count(*) nulls first) rnk
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
join hour
on rides.start_hour_key = hour.key
group by w.conditions, month_of_year
order by rnk
limit 10;

In [ ]:
%%sql
SELECT w.conditions, count(*) cnt, NTILE(4) over (order by count(*) nulls first) ntiles
FROM rides
JOIN weather AS w
ON w.key = rides.start_weather_key
group by w.conditions
order by ntiles;

In [ ]:
_.bar()

In [ ]:
%%sql
SELECT bike_key, hour datex, duration_ms
from rides
JOIN hour AS h
ON h.key = rides.start_hour_key
join bike
on bike.key=rides.bike_key
where bike_key=55 and hour< '2017-01-05'
order by bike_key, hour 

In [ ]:
%%sql
SELECT bike_key, hour datex, count(*) cnt, 
avg(sum(duration_ms)) over (order by bike_key, hour ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),
avg(sum(duration_ms)) over (order by bike_key, hour ROWS BETWEEN 1 PRECEDING AND 2 FOLLOWING)
FROM rides
JOIN hour AS h
ON h.key = rides.start_hour_key
join bike
on bike.key=rides.bike_key
where bike_key=55 and hour< '2017-01-05'
group by bike_key, hour;

In [ ]:
%%sql
SELECT bike_key, hour datex, count(*) cnt, 
LAG(hour,1,hour)  over (order by bike_key, hour) lagx,
LEAD(hour,1,hour) over (order by bike_key, hour) leadx,
cast(hour as timestamp)- cast(LAG(hour,1,hour)  over (order by bike_key, hour) as timestamp) diff_lag,
cast( LEAD(hour,1,hour) over (order by bike_key, hour) as timestamp)- cast(hour as timestamp) diff_lead
FROM rides
JOIN hour AS h
ON h.key = rides.start_hour_key
join bike
on bike.key=rides.bike_key
where bike_key=55 and hour< '2017-01-05'
group by bike_key, hour;